# Testing Cellmage with Gemini 2.5 Flash

This notebook demonstrates how to use the cellmage library with Google's Gemini 2.5 Flash model, exploring its capabilities and performance characteristics.

**Date:** April 24, 2025

## What is Gemini 2.5 Flash?

Gemini 2.5 Flash is a lightweight version of Google's Gemini 2.5 model designed for low-latency applications. It offers:
- Very fast response times
- Cost-effective inference
- Strong performance on a range of tasks
- Multimodal capabilities
- Effective tool use and function calling

## Setup & Installation

Let's set up our environment to work with the Gemini 2.5 Flash model.

In [1]:
# Setup environment
import os
import sys
import logging

# Skip dotenv loading for testing
os.environ["CELLMAGE_SKIP_DOTENV"] = "1"

# Set up logging
logging.basicConfig(level=logging.INFO)

# Ensure the cellmage package can be imported
# Get the absolute path of the current working directory
notebook_dir = os.getcwd()
# Get the project root directory (parent of the notebook directory)
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

print(f"Notebook directory: {notebook_dir}")
print(f"Project root directory: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added path: {project_root}")

try:
    # Import cellmage
    import cellmage

    # Check version - handle case where __version__ might not be available
    try:
        print(f"Cellmage version: {cellmage.__version__}")
    except AttributeError:
        print("Cellmage imported successfully, but version information is not available")
except ModuleNotFoundError as e:
    print(f"Error importing cellmage: {e}")
    print("\nDebug information:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Python path: {sys.path}")
    print("\nTry running this notebook from the project root directory")

2025-04-24 06:25:21,292 - cellmage - INFO - Cellmage logging initialized


Notebook directory: /Users/tpinto/madpin/cellmage/notebooks
Project root directory: /Users/tpinto/madpin/cellmage
Added path: /Users/tpinto/madpin/cellmage
Cellmage version: 0.1.0


## Setting Up the Gemini 2.5 Flash Client

We'll configure our LLM client to use Google's Gemini 2.5 Flash model.

In [2]:
from cellmage.adapters.direct_client import DirectLLMAdapter

# Create an LLM client with Gemini 2.5 Flash model
llm_client = DirectLLMAdapter(default_model="gemini-2.5-flash")

# You'll need to ensure your API key is set up in your environment
# For Google: export GOOGLE_API_KEY=your_api_key

# Check available models (optional)
available_models = llm_client.get_available_models()
print(f"Available models: {[m['id'] for m in available_models if 'id' in m][:5]}...")

# Get model info
model_info = llm_client.get_model_info("gemini-2.5-flash")
if model_info:
    print(f"Model info: {model_info}")
else:
    print("Model info not available")

2025-04-24 06:25:21,410 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_key' = sk-L...xxmA
2025-04-24 06:25:21,410 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_base' = https://litellm.oracle.madpin.dev
2025-04-24 06:25:21,411 - cellmage.adapters.direct_client - INFO - [Override] Setting 'model' = gemini-2.5-flash
2025-04-24 06:25:21,624 - cellmage.adapters.direct_client - INFO - Successfully fetched 45 models
2025-04-24 06:25:21,770 - cellmage.adapters.direct_client - ERROR - Error fetching model info for gemini-2.5-flash: 404 Client Error: Not Found for url: https://litellm.oracle.madpin.dev/v1/models/gemini-2.5-flash


Available models: ['o4-mini-high', 'llama-4-scout-17b-16e-instruct', 'embeddings', 'o3', 'Qwen2.5-72B-Instruct']...
Model info not available


## Creating a Chat Manager for Gemini 2.5 Flash

Let's set up a ChatManager specifically configured for Gemini 2.5 Flash.

In [3]:
# Create components for the chat manager
from cellmage.resources.memory_loader import MemoryLoader
from cellmage.storage.memory_store import MemoryStore

# Create in-memory components
persona_loader = MemoryLoader()
snippet_provider = MemoryLoader()
history_store = MemoryStore()

# Create specialized personas for Gemini 2.5 Flash
persona_loader.add_persona(
    name="creative_writer",
    system_message="You are a creative writing assistant with expertise in storytelling, poetry, and creative content generation. Provide imaginative and engaging responses.",
    config={
        "temperature": 0.8,  # Higher temperature for creativity
        "top_p": 0.95,  # Diverse sampling
    },
)

persona_loader.add_persona(
    name="data_analyst",
    system_message="You are a data analysis expert who provides insights on data processing, visualization, and statistical analysis using Python libraries like pandas, matplotlib, and scipy. Focus on clear, structured explanations.",
    config={
        "temperature": 0.3,  # Lower temperature for technical content
        "top_k": 40,  # Narrower sampling for technical precision
    },
)

# Create a chat manager
chat_manager = cellmage.ChatManager(
    llm_client=llm_client,
    persona_loader=persona_loader,
    snippet_provider=snippet_provider,
    history_store=history_store,
)

# List available personas
personas = chat_manager.list_personas()
print(f"Available personas: {personas}")

2025-04-24 06:25:21,814 - cellmage.resources.memory_loader - INFO - Added persona 'creative_writer' to memory
2025-04-24 06:25:21,814 - cellmage.resources.memory_loader - INFO - Added persona 'data_analyst' to memory
2025-04-24 06:25:21,815 - cellmage.chat_manager - INFO - Initializing ChatManager
2025-04-24 06:25:21,815 - cellmage.chat_manager - INFO - ChatManager initialized


Available personas: ['creative_writer', 'data_analyst']


## Testing Creative Writing Capabilities

Let's test how Gemini 2.5 Flash performs with creative writing tasks.

In [4]:
# Set creative writer persona
chat_manager.set_default_persona("creative_writer")

# Test creative writing prompts
creative_prompts = [
    "Write a short poem about the changing seasons",
    "Create a brief story opening that takes place in a futuristic underwater city",
    "Describe a fantasy creature that combines elements of three different animals",
]

# Test each prompt
for prompt in creative_prompts:
    print(f"\nPrompt: {prompt}")
    print("-" * 50)
    response = chat_manager.chat(prompt, stream=True)  # Using streaming for creative content
    print("-" * 50)

2025-04-24 06:25:21,820 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.8
2025-04-24 06:25:21,821 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 06:25:21,821 - cellmage.chat_manager - INFO - Default persona set to 'creative_writer'
2025-04-24 06:25:21,822 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:25:21,823 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 2 messages



Prompt: Write a short poem about the changing seasons
--------------------------------------------------


2025-04-24 06:25:27,524 - cellmage.chat_manager - INFO - Sending message to LLM with 4 messages in context
2025-04-24 06:25:27,524 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 4 messages


--------------------------------------------------

Prompt: Create a brief story opening that takes place in a futuristic underwater city
--------------------------------------------------


2025-04-24 06:25:33,369 - cellmage.chat_manager - INFO - Sending message to LLM with 6 messages in context
2025-04-24 06:25:33,369 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 6 messages


--------------------------------------------------

Prompt: Describe a fantasy creature that combines elements of three different animals
--------------------------------------------------
--------------------------------------------------


## Testing Data Analysis Capabilities

Now let's test how Gemini 2.5 Flash handles data analysis tasks.

In [5]:
# Set data analyst persona
chat_manager.set_default_persona("data_analyst")

# Add a pandas code snippet for context
snippet_provider.add_snippet(
    name="pandas_example",
    content="""```python
# Sample pandas dataframe
import pandas as pd
import numpy as np

# Create sample data
data = {
    'date': pd.date_range('2023-01-01', periods=10),
    'value': np.random.normal(100, 15, 10),
    'category': np.random.choice(['A', 'B', 'C'], 10)
}

df = pd.DataFrame(data)
print(df.head())
```""",
)

# Add the snippet to the conversation
chat_manager.add_snippet("pandas_example")

# Test data analysis questions
data_questions = [
    "How would I calculate the average value by category in this dataframe?",
    "Show me how to create a time series plot of the 'value' column over time",
    "What's the best way to detect and handle outliers in the 'value' column?",
]

# Test each question
for question in data_questions:
    print(f"\nQuestion: {question}")
    print("-" * 50)
    response = chat_manager.chat(question, stream=True)
    print("-" * 50)

2025-04-24 06:25:47,410 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:25:47,411 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_k' = 40
2025-04-24 06:25:47,411 - cellmage.chat_manager - INFO - Default persona set to 'data_analyst'
2025-04-24 06:25:47,412 - cellmage.resources.memory_loader - INFO - Added snippet 'pandas_example' to memory
2025-04-24 06:25:47,412 - cellmage.chat_manager - INFO - Added snippet 'pandas_example' as system message
2025-04-24 06:25:47,413 - cellmage.chat_manager - INFO - Sending message to LLM with 9 messages in context
2025-04-24 06:25:47,415 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 9 messages



Question: How would I calculate the average value by category in this dataframe?
--------------------------------------------------


2025-04-24 06:26:01,600 - cellmage.chat_manager - INFO - Sending message to LLM with 11 messages in context
2025-04-24 06:26:01,601 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 11 messages


--------------------------------------------------

Question: Show me how to create a time series plot of the 'value' column over time
--------------------------------------------------


2025-04-24 06:26:07,513 - cellmage.chat_manager - INFO - Sending message to LLM with 13 messages in context
2025-04-24 06:26:07,513 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 13 messages


--------------------------------------------------

Question: What's the best way to detect and handle outliers in the 'value' column?
--------------------------------------------------
--------------------------------------------------


## Testing Multimodal Capabilities

Gemini 2.5 Flash has multimodal capabilities. Let's test how it handles image descriptions and analysis.

In [6]:
# Note: Multimodal support requires specific setup with the LiteLLM adapter
# This is a demonstration of how it would work if properly configured

# For a proper implementation, you would need to extend the LiteLLM adapter
# to handle image inputs properly

print("Multimodal testing with Gemini 2.5 Flash:")
print("Note: This requires extending cellmage to support image inputs")

# Example of what the implementation might look like:
"""
# Create an image description
image_description = "[This is a placeholder for an image of a data visualization showing a scatter plot with clusters of data points in different colors.]"

# Add the image description as a special message
chat_manager.chat(f"Please analyze this data visualization: {image_description}", stream=True)
"""

Multimodal testing with Gemini 2.5 Flash:
Note: This requires extending cellmage to support image inputs


'\n# Create an image description\nimage_description = "[This is a placeholder for an image of a data visualization showing a scatter plot with clusters of data points in different colors.]"\n\n# Add the image description as a special message\nchat_manager.chat(f"Please analyze this data visualization: {image_description}", stream=True)\n'

## Testing Function Calling

Gemini 2.5 Flash has strong function calling capabilities. Let's test how it works with cellmage.

In [7]:
# Note: Function calling requires specific setup with the LiteLLM adapter
# This is a demonstration of how it would work conceptually

# Example function definition schema
function_definitions = [
    {
        "name": "get_weather",
        "description": "Get the current weather for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "The city and state or country"},
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The unit of temperature",
                },
            },
            "required": ["location"],
        },
    }
]

print("Function calling with Gemini 2.5 Flash:")
print("Note: This requires extending cellmage to support function definitions and execution")

# Example of what the implementation might look like:
"""
# Extended LiteLLM adapter with function calling support
llm_client_with_functions = LiteLLMAdapter(
    model="gemini-2.5-flash",
    functions=function_definitions
)

# Create a new chat manager with function support
function_chat_manager = cellmage.ChatManager(
    llm_client=llm_client_with_functions,
    persona_loader=persona_loader,
    snippet_provider=snippet_provider,
    history_store=history_store
)

# Test function calling
response = function_chat_manager.chat("What's the weather like in San Francisco?", stream=False)
"""

Function calling with Gemini 2.5 Flash:
Note: This requires extending cellmage to support function definitions and execution


'\n# Extended LiteLLM adapter with function calling support\nllm_client_with_functions = LiteLLMAdapter(\n    model="gemini-2.5-flash",\n    functions=function_definitions\n)\n\n# Create a new chat manager with function support\nfunction_chat_manager = cellmage.ChatManager(\n    llm_client=llm_client_with_functions,\n    persona_loader=persona_loader,\n    snippet_provider=snippet_provider,\n    history_store=history_store\n)\n\n# Test function calling\nresponse = function_chat_manager.chat("What\'s the weather like in San Francisco?", stream=False)\n'

## Performance Analysis

Let's analyze the performance of Gemini 2.5 Flash in terms of response time and token efficiency.

In [8]:
import time
import statistics

# Create a new conversation with clear history
chat_manager.clear_history()

# Prepare test prompts of varying complexity
test_prompts = [
    "What is the capital of France?",  # Simple fact
    "Explain how neural networks learn",  # Moderate explanation
    "Compare and contrast different sorting algorithms in terms of time complexity and use cases",  # Complex analysis
]

# Set a consistent persona
chat_manager.set_default_persona("data_analyst")

# Track response times
response_times = []
response_lengths = []

# Test each prompt and measure response time
for i, prompt in enumerate(test_prompts):
    print(f"\nPrompt {i + 1}: {prompt}")
    print("-" * 50)

    # Measure time
    start_time = time.time()
    response = chat_manager.chat(
        prompt, stream=False, add_to_history=False
    )  # Don't add to history to keep tests independent
    end_time = time.time()

    # Calculate response time and length
    response_time = end_time - start_time
    response_times.append(response_time)
    response_lengths.append(len(response))

    print(f"Response time: {response_time:.2f} seconds")
    print(f"Response length: {len(response)} characters")
    print("-" * 50)

# Calculate statistics
if response_times and response_lengths:
    avg_time = statistics.mean(response_times)
    avg_length = statistics.mean(response_lengths)
    chars_per_second = sum(response_lengths) / sum(response_times)

    print(f"\nPerformance Summary:")
    print(f"Average response time: {avg_time:.2f} seconds")
    print(f"Average response length: {avg_length:.2f} characters")
    print(f"Characters per second: {chars_per_second:.2f}")

2025-04-24 06:26:26,691 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:26:26,692 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:26:26,693 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_k' = 40
2025-04-24 06:26:26,693 - cellmage.chat_manager - INFO - Default persona set to 'data_analyst'
2025-04-24 06:26:26,694 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:26:26,694 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 2 messages



Prompt 1: What is the capital of France?
--------------------------------------------------


2025-04-24 06:26:29,268 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:26:29,268 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 2 messages


Response time: 2.57 seconds
Response length: 35 characters
--------------------------------------------------

Prompt 2: Explain how neural networks learn
--------------------------------------------------


2025-04-24 06:26:44,610 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:26:44,614 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 2 messages


Response time: 15.34 seconds
Response length: 5282 characters
--------------------------------------------------

Prompt 3: Compare and contrast different sorting algorithms in terms of time complexity and use cases
--------------------------------------------------
Response time: 31.13 seconds
Response length: 14702 characters
--------------------------------------------------

Performance Summary:
Average response time: 16.35 seconds
Average response length: 6673.00 characters
Characters per second: 408.17


## Comparing Responses Across Different Prompts

Let's test how Gemini 2.5 Flash handles the same prompts with different personas to evaluate consistency and adaptability.

In [9]:
# Create a new conversation with clear history
chat_manager.clear_history()

# Define a consistent test prompt
test_prompt = "Explain the concept of machine learning to a beginner"

# Test with different personas
personas = ["creative_writer", "data_analyst"]

for persona in personas:
    print(f"\nTesting with persona: {persona}")
    print("-" * 50)

    # Set the persona
    chat_manager.set_default_persona(persona)
    chat_manager.clear_history(keep_system=True)  # Keep only the system message

    # Send the prompt
    response = chat_manager.chat(test_prompt, stream=False)

    # Print a summary of the response (first 300 characters)
    print(f"Response summary: {response[:300]}...")
    print("-" * 50)

2025-04-24 06:27:15,754 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:27:15,761 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.8
2025-04-24 06:27:15,762 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 06:27:15,763 - cellmage.chat_manager - INFO - Default persona set to 'creative_writer'
2025-04-24 06:27:15,767 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:27:15,772 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:27:15,781 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 3 messages



Testing with persona: creative_writer
--------------------------------------------------


2025-04-24 06:27:28,606 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:27:28,607 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_k' = 40
2025-04-24 06:27:28,607 - cellmage.chat_manager - INFO - Default persona set to 'data_analyst'
2025-04-24 06:27:28,608 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:27:28,608 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:27:28,608 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 3 messages


Response summary: Okay, let's break down machine learning in a simple way, like teaching something new.

Imagine you want to teach a computer to recognize cats and dogs in pictures.

**The "Old Way" (Traditional Programming):**

You would have to look at cats and dogs yourself and write down a huge list of rules for ...
--------------------------------------------------

Testing with persona: data_analyst
--------------------------------------------------
Response summary: Okay, imagine you want to teach a computer to do something, but instead of giving it a step-by-step instruction manual for *every single possible situation*, you want it to *figure things out* on its own by looking at examples.

That's the core idea behind **Machine Learning**.

Think about how *you...
--------------------------------------------------


## Saving and Loading Conversations

Let's test saving and loading conversations with Gemini 2.5 Flash.

In [10]:
# Create a new conversation with the creative writer persona
chat_manager.clear_history()
chat_manager.set_default_persona("creative_writer")

# Have a brief conversation about storytelling
print("Creating a new conversation about storytelling...")
chat_manager.chat("What are the key elements of a compelling story?", stream=False)
chat_manager.chat("How do I develop interesting characters?", stream=False)

# Save the conversation
save_path = chat_manager.save_conversation("gemini25flash_storytelling_convo")
print(f"\nConversation saved to: {save_path}")

# Clear the history
chat_manager.clear_history()
print(f"History cleared, current message count: {len(chat_manager.get_history())}")

# Load the conversation back
if save_path:
    chat_manager.load_conversation(save_path)
    print(f"Conversation loaded, message count: {len(chat_manager.get_history())}")

    # Display the loaded conversation
    print("\nLoaded conversation:")
    for i, message in enumerate(chat_manager.get_history()):
        if message.role != "system":  # Skip system message for brevity
            print(f"{i}. {message.role}: {message.content[:100]}...")

2025-04-24 06:27:39,108 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:27:39,110 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.8
2025-04-24 06:27:39,111 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 06:27:39,111 - cellmage.chat_manager - INFO - Default persona set to 'creative_writer'
2025-04-24 06:27:39,112 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:27:39,112 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 3 messages


Creating a new conversation about storytelling...


2025-04-24 06:27:53,385 - cellmage.chat_manager - INFO - Sending message to LLM with 5 messages in context
2025-04-24 06:27:53,386 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 5 messages
2025-04-24 06:28:09,273 - cellmage.storage.memory_store - INFO - Saved conversation to memory with ID: gemini25flash_storytelling_convo
2025-04-24 06:28:09,274 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:28:09,275 - cellmage.history_manager - INFO - Loaded conversation from gemini25flash_storytelling_convo with 6 messages



Conversation saved to: gemini25flash_storytelling_convo
History cleared, current message count: 2
Conversation loaded, message count: 6

Loaded conversation:
2. user: What are the key elements of a compelling story?...
3. assistant: Ah, the alchemical blend that turns mere words into worlds and simple sequences into soul-stirring j...
4. user: How do I develop interesting characters?...
5. assistant: Ah, breathing life into characters! This is arguably one of the most rewarding and crucial aspects o...


## Conclusion: Gemini 2.5 Flash Performance

This notebook has demonstrated the capabilities of Gemini 2.5 Flash when used with the cellmage library:

1. **Response Speed**: Gemini 2.5 Flash generally provides very quick responses, making it excellent for interactive applications.
2. **Creative Content**: It performs well on creative tasks with appropriate temperature settings.
3. **Technical Accuracy**: With lower temperature settings, it can provide accurate technical information.
4. **Multimodal Potential**: While requiring additional implementation, its multimodal capabilities offer interesting possibilities.
5. **Function Calling**: Its function calling capabilities can be leveraged with extensions to the cellmage library.

**Use Cases for Gemini 2.5 Flash:**
- Interactive chatbots requiring low latency
- Creative content generation
- Technical support and documentation
- Data analysis explanations
- Applications with budget constraints

Gemini 2.5 Flash offers an excellent balance of performance, speed, and cost, making it a versatile option for many applications.